In [5]:
from docx import Document
from docx.shared import Pt
from docx.enum.text import WD_ALIGN_PARAGRAPH

import easygui

In [6]:
import pandas as pd
cp_raw = pd.read_excel(easygui.fileopenbox(msg="Select the MIS Case Plan export file"))

cp_raw.columns

Index(['date', 'hh', 'hh_member', 'Beneficiary type', 'SPECIFIC', 'domain',
       'goal', 'gap', 'priority', 'service', 'responsible_person',
       'external_person', 'timeline', 'result', 'date_achieved', 'reason',
       'county', 'subcounty', 'ward', 'lip', 'Caregiver', 'CHV'],
      dtype='object')

In [7]:
#Data Pre-processing tasks  --------------------------------4245087

# Columns to format and replace NaN/NaT values
columns_to_format = ['timeline', 'date_achieved', 'date']
columns_to_replace = ['timeline', 'result', 'date_achieved', 'reason']

# Replace NaN or NaT values with blank (empty string) in specific columns
cp_raw[columns_to_replace] = cp_raw[columns_to_replace].fillna('')

# Format date columns as "YYYY-MM-DD" if they have a valid date value, otherwise set them to ''
for column in columns_to_format:
    cp_raw[column] = cp_raw[column].apply(lambda x: x.strftime("%Y-%m-%d") if isinstance(x, pd.Timestamp) else '')

# Replace "GOAL 2" in the 'goal' column
cp_raw['goal'] = cp_raw['goal'].replace({"GOAL 2: GH 2 HE. Increase HIV treatment adherence, continuity of treatment and viral suppression NB: Documented viral load results exist in health facility records where caregiver, child, adolescent access care and treatment": "GOAL 2: GH 2 HE. Increase HIV treatment adherence, continuity of treatment and viral suppression"})

import os
import pdfkit

def exportHH(cp, docX):
    # Get the tables in the document
    table1 = docX.tables[0]
    
    data = cp
    
    # Check if DataFrame is empty
    if not data.empty:
        # Get the first row
        header = data.head(1)[['date', 'hh','county', 'subcounty', 'ward', 'lip', 'Caregiver', 'CHV']]

        # Modify cell value [row, column]
        if not header.empty:
            table1.cell(0, 1).text = str(header['county'].values[0])
            table1.cell(0, 3).text = str(header['subcounty'].values[0])
            table1.cell(0, 5).text = str(header['ward'].values[0])
            table1.cell(1, 1).text = str(header['Caregiver'].values[0])
            table1.cell(1, 3).text = str(header['hh'].values[0])  # Assuming 'hh' column is correct
            table1.cell(1, 5).text = str(header['CHV'].values[0])
            table1.cell(2, 1).text = str(header['date'].values[0])
            table1.cell(2, 3).text = str(header['lip'].values[0])

            household_id = str(header['hh'].values[0])+"_"+str(header["lip"].values[0].replace("/","_"))+"_"+str(header['subcounty'].values[0].replace("/","_"))
            table = docX.tables[1]
            
            # Clear existing data rows from the table
            for row in table.rows[1:]:
                table._element.remove(row._element)
                
            for index, row in data.iterrows():
                row_data = row[['hh_member', 'goal', 'gap', 'service', 'external_person', 'timeline', 'result', 'date_achieved', 'reason']]
                # Add a new row to the table
                new_row = table.add_row().cells
                # Iterate over each cell in the row and set its value
                for i, cell_value in enumerate(row_data):
                    new_row[i].text = str(cell_value)

            # Add 5 extra rows
            for _ in range(5):
                table.add_row()
            
            # Define the output directory
            output_dir = os.path.join(os.getcwd(), "caseplan_output")

            # Create the output directory if it doesn't exist
            if not os.path.exists(output_dir):
                os.makedirs(output_dir)

            # Save the modified document in the output directory as DOCX
            docx_output_path = os.path.join(output_dir, household_id+'_Case_Plan_Printout.docx')
            docX.save(docx_output_path)

            # Convert the DOCX file to PDF
            pdf_output_path = os.path.join(output_dir, household_id+'_Case_Plan_Printout.pdf')
            #pdfkit.from_file(docx_output_path, pdf_output_path)
            
            del docX, table, table1
        else:
            print("DataFrame 'cp' has no rows.")
    else:
        print("DataFrame 'cp' is empty.")


In [10]:
# Open an existing document
doc = Document(easygui.fileopenbox(msg="Select word template"))

# Get the unique values from the column containing hh values
unique_hh = cp_raw["hh"].unique()

for hh in unique_hh:
    cpX = cp_raw[cp_raw["hh"] == hh].copy()  # Make a copy of the filtered DataFrame
    print("Preparing Caseplan for hh: {}; Rows: {}".format(hh, cpX.shape[0]))
    
    exportHH(cpX, doc)


Preparing Caseplan for hh: 4378549; Rows: 3
Preparing Caseplan for hh: 1885701; Rows: 3
Preparing Caseplan for hh: 1865196; Rows: 2
Preparing Caseplan for hh: 1804991; Rows: 5
Preparing Caseplan for hh: 4187021; Rows: 1
Preparing Caseplan for hh: 3762703; Rows: 4
Preparing Caseplan for hh: 1771231; Rows: 2
Preparing Caseplan for hh: 3448605; Rows: 2
Preparing Caseplan for hh: 4187103; Rows: 2
Preparing Caseplan for hh: 2295665; Rows: 6
Preparing Caseplan for hh: 1781267; Rows: 1
Preparing Caseplan for hh: 4531599; Rows: 1
Preparing Caseplan for hh: 3544093; Rows: 8
Preparing Caseplan for hh: 4378565; Rows: 15
Preparing Caseplan for hh: 4288321; Rows: 7
Preparing Caseplan for hh: 1685152; Rows: 9
Preparing Caseplan for hh: 1656176; Rows: 4
Preparing Caseplan for hh: 4297827; Rows: 4
Preparing Caseplan for hh: 3472483; Rows: 8
Preparing Caseplan for hh: 3464709; Rows: 10
Preparing Caseplan for hh: 4202965; Rows: 8
Preparing Caseplan for hh: 1770333; Rows: 9
Preparing Caseplan for hh: 378